# Model Deployment: From Jupyter to Production

> **"A model is only as good as its deployment."**

## Learning Objectives
- Learn how to deploy machine learning models to production
- Master model serialization and API development
- Understand model monitoring and performance tracking
- Implement containerization and cloud deployment
- Apply MLOps best practices for production systems


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import joblib
import json
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("Libraries imported successfully!")


## 1. Model Deployment Fundamentals

### What is Model Deployment?
Model deployment is the process of making a trained machine learning model available for use in production environments. It involves packaging the model, creating APIs, and ensuring the model can handle real-world data.

### Key Components

#### 1. Model Serialization
- **Pickle**: Python's built-in serialization
- **Joblib**: Optimized for NumPy arrays
- **ONNX**: Cross-platform model format
- **TensorFlow SavedModel**: For TensorFlow models

#### 2. API Development
- **REST APIs**: HTTP-based interfaces
- **GraphQL**: Flexible query language
- **gRPC**: High-performance RPC framework

#### 3. Monitoring and Maintenance
- **Performance Monitoring**: Track accuracy, latency
- **Data Drift Detection**: Monitor input distribution changes
- **Model Retraining**: Update models with new data


In [ ]:
# Generate sample dataset for model deployment
np.random.seed(42)

# Create synthetic dataset
n_samples = 1000
n_features = 10

# Generate features
X = np.random.randn(n_samples, n_features)
feature_names = [f'feature_{i}' for i in range(n_features)]

# Create target variable
true_coeffs = np.array([2.0, 1.5, -0.8, 0, 0, 0, 0, 0, 0, 0])
y = X @ true_coeffs + np.random.normal(0, 0.5, n_samples)
y_binary = (y > np.median(y)).astype(int)

# Create DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y_binary

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dataset Overview:")
print("=" * 50)
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")
print(f"Target distribution: {np.bincount(y_train)}")

# Train multiple models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

trained_models = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    trained_models[name] = {
        'model': model,
        'accuracy': accuracy,
        'predictions': y_pred
    }
    
    print(f"{name} Accuracy: {accuracy:.3f}")

# Visualize model performance
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Accuracy comparison
model_names = list(trained_models.keys())
accuracies = [trained_models[name]['accuracy'] for name in model_names]

bars = axes[0].bar(model_names, accuracies, alpha=0.7)
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Performance Comparison')
axes[0].grid(True, alpha=0.3)

# Add value labels
for bar, accuracy in zip(bars, accuracies):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{accuracy:.3f}', ha='center', va='bottom')

# Confusion matrix for best model
best_model_name = max(trained_models.keys(), key=lambda x: trained_models[x]['accuracy'])
best_model = trained_models[best_model_name]['model']
y_pred_best = trained_models[best_model_name]['predictions']

cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])
axes[1].set_title(f'Confusion Matrix - {best_model_name}')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()


In [ ]:
# Model Serialization and Deployment
print("Model Serialization and Deployment:")
print("=" * 50)

# Select best model for deployment
best_model_name = max(trained_models.keys(), key=lambda x: trained_models[x]['accuracy'])
best_model = trained_models[best_model_name]['model']

print(f"Selected model for deployment: {best_model_name}")
print(f"Model accuracy: {trained_models[best_model_name]['accuracy']:.3f}")

# Serialize model and scaler
model_filename = 'best_model.joblib'
scaler_filename = 'scaler.joblib'

joblib.dump(best_model, model_filename)
joblib.dump(scaler, scaler_filename)

print(f"\nModel saved to: {model_filename}")
print(f"Scaler saved to: {scaler_filename}")

# Create model metadata
model_metadata = {
    'model_name': best_model_name,
    'model_type': type(best_model).__name__,
    'accuracy': float(trained_models[best_model_name]['accuracy']),
    'feature_names': feature_names,
    'n_features': len(feature_names),
    'training_date': '2024-01-01',
    'version': '1.0.0'
}

# Save metadata
with open('model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)

print(f"Model metadata saved to: model_metadata.json")

# Load and test serialized model
print("\nTesting serialized model:")
print("-" * 30)

# Load model and scaler
loaded_model = joblib.load(model_filename)
loaded_scaler = joblib.load(scaler_filename)

# Test prediction
test_sample = X_test_scaled[0:1]  # First test sample
prediction = loaded_model.predict(test_sample)
probability = loaded_model.predict_proba(test_sample)

print(f"Test sample prediction: {prediction[0]}")
print(f"Prediction probability: {probability[0]}")
print(f"Original accuracy: {trained_models[best_model_name]['accuracy']:.3f}")

# Verify model consistency
all_predictions = loaded_model.predict(X_test_scaled)
original_predictions = trained_models[best_model_name]['predictions']
predictions_match = np.array_equal(all_predictions, original_predictions)

print(f"Predictions match original: {predictions_match}")

# Model deployment simulation
print("\nModel Deployment Simulation:")
print("-" * 30)

class ModelPredictor:
    """Simple model predictor class for deployment."""
    
    def __init__(self, model_path, scaler_path, metadata_path):
        self.model = joblib.load(model_path)
        self.scaler = joblib.load(scaler_path)
        
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
    
    def predict(self, features):
        """Make prediction on new data."""
        # Validate input
        if len(features) != self.metadata['n_features']:
            raise ValueError(f"Expected {self.metadata['n_features']} features, got {len(features)}")
        
        # Scale features
        features_scaled = self.scaler.transform([features])
        
        # Make prediction
        prediction = self.model.predict(features_scaled)[0]
        probability = self.model.predict_proba(features_scaled)[0]
        
        return {
            'prediction': int(prediction),
            'probability': float(probability[1]),  # Probability of class 1
            'confidence': float(max(probability))
        }
    
    def get_model_info(self):
        """Get model information."""
        return self.metadata

# Initialize predictor
predictor = ModelPredictor(model_filename, scaler_filename, 'model_metadata.json')

# Test with new data
print("Testing with new data:")
print("-" * 20)

# Generate new sample
new_sample = np.random.randn(10)  # 10 features
result = predictor.predict(new_sample)

print(f"New sample: {new_sample}")
print(f"Prediction: {result['prediction']}")
print(f"Probability: {result['probability']:.3f}")
print(f"Confidence: {result['confidence']:.3f}")

# Model information
print(f"\nModel Information:")
print("-" * 20)
model_info = predictor.get_model_info()
for key, value in model_info.items():
    print(f"{key}: {value}")

# Performance monitoring simulation
print(f"\nPerformance Monitoring:")
print("-" * 30)

# Simulate monitoring over time
n_days = 30
daily_accuracy = []
daily_predictions = []

for day in range(n_days):
    # Simulate daily predictions
    n_predictions = np.random.randint(50, 200)
    daily_pred = []
    
    for _ in range(n_predictions):
        # Generate random sample
        sample = np.random.randn(10)
        pred = predictor.predict(sample)
        daily_pred.append(pred['prediction'])
    
    # Simulate accuracy (with some noise)
    base_accuracy = trained_models[best_model_name]['accuracy']
    noise = np.random.normal(0, 0.05)
    daily_acc = max(0, min(1, base_accuracy + noise))
    
    daily_accuracy.append(daily_acc)
    daily_predictions.append(n_predictions)

# Plot monitoring results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Daily accuracy
axes[0].plot(range(1, n_days + 1), daily_accuracy, 'b-', alpha=0.7)
axes[0].axhline(y=base_accuracy, color='r', linestyle='--', label='Baseline Accuracy')
axes[0].set_xlabel('Day')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Daily Model Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Daily predictions
axes[1].bar(range(1, n_days + 1), daily_predictions, alpha=0.7)
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Number of Predictions')
axes[1].set_title('Daily Prediction Volume')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average daily accuracy: {np.mean(daily_accuracy):.3f}")
print(f"Average daily predictions: {np.mean(daily_predictions):.1f}")
print(f"Total predictions over {n_days} days: {sum(daily_predictions)}")
